# End-of-Module Assessment 2

## Instructions

Please read and follow these instructions and specifications carefully.  

1. This test is available from now until August 6 (Friday), 11:59 PM.
2. This test covers Module 5, which is about data analysis using libraries. However, you will need fundamental skills from Modules 2 to 4 to do this test properly.
3. This test is by project group. 
4. You are allowed to use the specified non-standard libraries for this test: `pandas`, `numpy`, and `matplotlib`. You are allowed to use any standard library for this test.
5. Answers to problems that are hard-coded, done in another program such as Excel, or go against the learning outcomes in any other manner will be given no credit. If you are unsure whether your approach to a problem goes against the learning outcomes, ask.  
6. We reserve the right to scrutinize answers to problems that significantly go against the philosophy of a library. For example, if you answer a Pandas item mostly using vanilla Python loops and data structures, you go against the philosophy of Pandas (and NumPy), which is to _vectorize_ your operations as much as you can. We reserve the right to scrutinize your answer accordingly.
7. You are encouraged to use only the provided code cell for each number. However, if you _need_ more cells, you may add them. If you do add more cells, make sure to label them very clearly. Code cells that are not clearly labelled will not be checked.
8. If you are asked to output files, please write them all to the `output` folder. If your kit does not have an `output` folder, you may make one.

Submission instructions for normal (i.e., not cross-section) groups:
1. Assign a group leader. The leader is who will submit the assessment.
2. The leader must create a NEW GitHub repository. This repository is what they will submit to the Canvas assignment to indicate that the group has submitted. They should name this repository SECTION-GROUPNUM-EOMA2 (e.g., A-5-EOMA2).
3. The leader must upload the contents of this folder to the repository. This includes the EOMA2.ipynb file, the output folder, and all section folders.
4. To officially submit the assignment, the group leader must upload the link to the GitHub repository and ALL GROUP MEMBERS' COAs to the Canvas assignment.

Submission instructions for cross-section groups:
1. Assign a group leader. The leader is who will create the GitHub repository, and they are who will add the required files to the GitHub repository. 
2. The leader must create a NEW GitHub repository. They should name this repository GROUPNAME-EOMA2 (e.g., CS1-EOMA2). 
3. The leader must upload the contents of this folder to the repository. This includes the EOMA2.ipynb file, the output folder, and all section folders.
4. Since your group is split across sections, your group must choose one member per represented section to submit the exact same file kit to their respective section's Canvas assignment. For example, if all sections (A, G, and H) are represented in your group, choose one person per section (i.e., 3 people) to submit your EOMA2 submission to their respective section's Canvas assignment. If only two sections are represented, then only 2 submissions (i.e., one per represented section, so in this case 2) must be submitted. Please note that we will quite literally not give a grade to students in cross-section groups who are not represented by a submission for their section. 


## Section 2: Cleaning Marketing Data (75 points)

### Context

You are a freelance data consultant. You were hired by a marketing firm that collects customer data for their clients, which are big fast-moving-consumer-goods (FMCG) firms. Your client, the marketing firm, currently handles their operations through a "multi-channel" approach. (To grossly oversimplify, a multi-channel approach to marketing means that a brand can interact with customers across more than one channel). However, they would like to upgrade their system to use an "omni-channel" approach. (An omni-channel approach means that a customer can be recognized across multiple channels. This is important because you will be able to personalize their experience better.)  

Your client currently has three main data sets:  
1. Manual interaction (`./section2/raw-manual.csv`)
2. Website interaction (`./section2/raw-website.csv`)
3. Email interaction (`./section2/raw-email.csv`)

They task you with cleaning and merging their data to help them establish their omni-channel capabilities.  

Again, you are to use Pandas for this, because your client's real databases are much, much bigger than these samples.

### 2.1

Your client first asks you to clean up their data. They want their data to follow these rules:  
1. Emails must not have any capital letters.  
2. Mobile numbers must be formatted as `+63XXXXXXXXXX` (e.g., +639174005892).
3. All the fields (fname, lname, email, and mobile) must be filled.
4. The fields must only contain data relevant to them; they must not contain data that does not belong in their column.

You have surveyed the data a little, and you have determined that though the data is dirty, it at least adheres to the following patterns:
1. All of the emails adhere to the email standard at https://emailregex.com/. (That's a hint.)
2. There are only 4 different "patterns" under the `mobile` columns per data set. (An empty field is not one of these 4 patterns).
3. The manual data set appears to have sometimes merged the `email` value and the `mobile` value into the `email` column and left the `mobile` column empty. The mobile value in these defective columns may be any one of the 4 different "patterns" that mobile numbers follow.  

Perform this cleanup on each of the three raw data sets.  

Save the cleaned data sets as `./output/cleaned-{dataset_name}.csv`. So, for example, `./section2/raw-manual.csv` will be cleaned and saved as `./output/cleaned-manual.csv`.  

Do not include the dataframe index in the output csv.

In [2]:
# CODE CELL
import pandas as pd
import numpy as np
import re # That's a hint.

# CODE START

# ------------------------------------------------- MANUAL ----------------------------------------------------

filename = "./section2/raw-manual.csv"
manual_df = pd.read_csv(filename)

#COLUMNS 
#convert to string
manual_df = manual_df.astype({"email": str})

#split the emails to new df
new_manual_df = manual_df['email'].str.split("/", n = 1, expand = True)

#copy new df columns to old df
manual_df['nemail']=new_manual_df[0]
manual_df['nmobile']=new_manual_df[1]

#remove NaN/NA and combine 2 mobile columns
manual_df['fmobile'] = manual_df['mobile'].fillna('') + manual_df['nmobile'].fillna('')

#remove extra columns
manual_df.drop(columns=['email', 'mobile', 'nmobile'], inplace=True)

#rename columns
manual_df['email']=manual_df['nemail']
manual_df['mobile']=manual_df['fmobile']

#rearrange columns
cols = ['email', 'fname', 'lname', 'mobile']
manual_df = manual_df[cols]

#EMAILS
#convert to string
manual_df = manual_df.astype({"email": 'string'})

#capitalize emails
manual_df['email']=manual_df['email'].str.replace(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",
                                              lambda x:x.group(0).lower())

#NUMBERS
manual_df.mobile = manual_df.mobile.str.replace(' ', '')
manual_df.mobile=np.where(manual_df.mobile.str.startswith('63'),'+' + manual_df.mobile,manual_df.mobile)
manual_df.mobile=np.where(manual_df.mobile.str.startswith('09'),'+63'+ 
manual_df.mobile.str[1:],manual_df.mobile)
manual_df.mobile=np.where(manual_df.mobile.str.startswith('9'),'+63'+ manual_df.mobile,manual_df.mobile)

#CSV FILE
manual_df.to_csv("./output/cleaned-manual.csv", index=False)


# ------------------------------------------------- WEBSITE ----------------------------------------------------
filename = "./section2/raw-website.csv"
website_df = pd.read_csv(filename)

#EMAILS
#Convert to String
website_df = website_df.astype({"email": 'string'})

#Capitalize Emails
website_df['email']=website_df['email'].str.replace(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",
                                              lambda x:x.group(0).lower())

#NUMBERS
website_df.mobile = website_df.mobile.str.replace(' ', '')
website_df.mobile=np.where(website_df.mobile.str.startswith('63'),'+' + website_df.mobile,website_df.mobile)
website_df.mobile=np.where(website_df.mobile.str.startswith('09'),'+63'+ 
website_df.mobile.str[1:],website_df.mobile)
website_df.mobile=np.where(website_df.mobile.str.startswith('9'),'+63'+ website_df.mobile,website_df.mobile)

#CSV 
website_df.to_csv("./output/cleaned-website.csv", index=False)

# ------------------------------------------------- EMAILS ----------------------------------------------------

filename = "./section2/raw-email.csv"
email_df = pd.read_csv(filename)
email_df  

#EMAILS
#Convert to String
email_df = email_df.astype({"email": 'string'})

#Capitalize Emails
email_df['email']=email_df['email'].str.replace(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",
                                              lambda x:x.group(0).lower())

#NUMBERS
email_df.mobile = email_df.mobile.str.replace(' ', '')
email_df.mobile=np.where(email_df.mobile.str.startswith('63'),'+' + email_df.mobile,email_df.mobile)
email_df.mobile=np.where(email_df.mobile.str.startswith('09'),'+63'+ 
email_df.mobile.str[1:],email_df.mobile)
email_df.mobile=np.where(email_df.mobile.str.startswith('9'),'+63'+ email_df.mobile,email_df.mobile)

#test 
email_df.to_csv("./output/cleaned-email.csv", index=False)

# CODE END

KeyError: 1

### 2.2

Your client wants you to generate a comprehensive list of the identities present in their scattered data. Their specifications are as follows:
1. Use the cleaned data from 2.1 as your data source.
2. Use the emails as the indicator of a unique identity.
3. Sort the final file by the email in alphabetical order.

Save the cleaned data sets as `./output/merged-identities.csv`.  

Do not include the dataframe index in the output csv.

In [7]:
# CODE CELL
import pandas as pd

# CODE START

#merge all csvs
merged_df = pd.concat(map(pd.read_csv, ["./output/cleaned-website.csv", "./output/cleaned-email.csv","./output/cleaned-manual.csv"]), ignore_index=True)

#remove duplicates
merged_df.sort_values("email", inplace = True)
merged_df.drop_duplicates(subset ="email",keep = 'first', inplace = True)

#alphabetical order
merged_df.sort_values('email', ascending=False)

#add + again
merged_df['mobile'] = merged_df['mobile'].astype(str)
merged_df.mobile = np.where(merged_df.mobile.str.startswith('63'),'+' + merged_df.mobile,merged_df.mobile)

#CSV
merged_df.to_csv("./output/merged-identities.csv", index=False)

# CODE END